In [ ]:
# Install PyTorch and TorchVision (Make sure you have the right versions compatible with your CUDA version)
!pip install torch torchvision

# Install Pillow (PIL)
!pip install Pillow

# Optional: Install OpenCV if you need more image processing capabilities
!pip install opencv-python-headless


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2
import PIL
from io import BytesIO
from google.colab.patches import cv2_imshow

# JavaScript to display the webcam feed and add a 'Capture' button
def js_capture_image():
    js = Javascript('''
    async function captureImage() {
        // Check if video and button elements already exist, if so, remove them
        const existingVideo = document.querySelector('video');
        const existingButton = document.querySelector('button');
        if (existingVideo) existingVideo.remove();
        if (existingButton) existingButton.remove();

        // Create video and button elements
        const video = document.createElement('video');
        const button = document.createElement('button');
        button.textContent = 'Capture';

        // Append video and button to the page
        document.body.appendChild(video);
        document.body.appendChild(button);

        // Set up webcam stream
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        video.style.width = '320px';
        await new Promise((resolve) => video.onloadedmetadata = resolve);
        video.play();

        // Wait for the 'Capture' button to be clicked
        await new Promise((resolve) => button.onclick = resolve);

        // Capture the image from the video stream
        const canvas = document.createElement('canvas');
        const context = canvas.getContext('2d');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        context.drawImage(video, 0, 0, canvas.width, canvas.height);

        // Stop the webcam and remove the video and button elements
        stream.getTracks().forEach(track => track.stop());
        video.remove();
        button.remove();

        // Return the captured image data
        return canvas.toDataURL('image/png');
    }
    captureImage();
    ''')
    display(js)

# Function to decode image and convert it to OpenCV format
def get_image():
    image_data = eval_js('captureImage()')
    image_bytes = b64decode(image_data.split(',')[1])
    image = np.array(PIL.Image.open(BytesIO(image_bytes)))
    return image

# Trigger the capture image process
js_capture_image()
image = get_image()

# Show the captured image
cv2_imshow(image)

# Save the image to /content/ (Colab file explorer section)
image_path = '/content/captured_image.png'
cv2.imwrite(image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Display a message confirming the saved image path
print(f"Image saved to: {image_path}")

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os

# Step 1: Load the saved model, ensuring compatibility with CPU
model = torch.load("/content/fruit.pth", map_location=torch.device('cpu'))
model.eval()  # Set the model to evaluation mode

# Step 2: Define the same transformations used during training
transformer = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the image to 128x128
    transforms.ToTensor()  # Convert the image to a tensor
])

# Step 3: Class names (make sure these match your training classes)
class_names = ['Apple_Healthy', 'AppleRotten', 'BananaHealthy', 'Banana_Rotten',
               'Bellpepper_Healthy', 'BellpepperRotten', 'Carrot_Healthy',
               'Carrot_Rotten', 'CucumberHealthy', 'CucumberRotten', 'Grape_Healthy',
               'Grape_Rotten', 'GuavaHealthy', 'GuavaRotten', 'Jujube_Healthy',
               'Jujube_Rotten', 'MangoHealthy', 'MangoRotten', 'Orange_Healthy',
               'Orange_Rotten', 'PomegranateHealthy', 'PomegranateRotten', 'Potato_Healthy',
               'Potato_Rotten', 'StrawberryHealthy', 'StrawberryRotten', 'Tomato_Healthy',
               'Tomato__Rotten']

# Step 4: Function to predict class of a single image
def predict_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path)  # Open the image
    image = transformer(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension (1, C, H, W)

    # Perform the prediction on CPU
    with torch.no_grad():
        output = model(image)  # Forward pass through the model
        _, predicted = torch.max(output, 1)  # Get the index of the max log-probability

    # Get the class label
    predicted_class = class_names[predicted.item()]

    # Output the prediction
    print(f"Image: {os.path.basename(image_path)} -> Predicted Class: {predicted_class}")

# Step 5: Test the model with a sample image
image_path =   "/content/captured_image.png"  # Update this path
predict_image(image_path)

Image: ber-1000x1000.jpeg -> Predicted Class: Jujube_Healthy


<ipython-input-29-1420ecb2cefb>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/fruit.pth", map_location=torch.device('cpu'))
